In [154]:
from sympy import Ellipse, Point, Rational
import itertools
from scipy.optimize import minimize
from sympy.solvers import solve
from sympy import *

In [189]:
def ellipse_base(b1, b2, rssi1, rssi2):
    # ellipse de la forme X^2 + Y^2 + x_coeff*X + y_coeff*Y +  const_coeff = 0
    # bi sont les coordonnées des bases i
    x_coeff = (-2*rssi2/(rssi2-rssi1))*(b1[0]-b2[0]*rssi2/rssi1)
    y_coeff = (-2*rssi2/(rssi2-rssi1))*(b1[1]-b2[1]*rssi2/rssi1)
    const_coeff = ((b1[0]+b1[1])*rssi2 + (b2[0]+b2[1])*rssi1)/(rssi2-rssi1)
    return [x_coeff, y_coeff, const_coeff]

In [222]:
def distance(ellipse, X):
    # calcul de la plus petite distance entre le point X et l'ellipse d'equation type ellipse_base
    # on utilisera le solveur pour obtenir le point de l'ellipse apres avoir dérivé le schoulblique
    x2 = Symbol('x_2', real=True)
    y2 = Symbol('y_2', real=True)
    x = X[0]
    y = X[1]
    distance = 0
    eq = diff((x2 - x)**2 + (y2 - y)**2, x2)
    print(eq)
    print(x2**2 + y2**2 + ellipse[0] * x2 + ellipse[1] * y2 + ellipse[2])
    sol = solve((eq, x2**2 + y2**2 + ellipse[0] * x2 + ellipse[1] * y2 + ellipse[2]), x2, y2)
    for i in range(len(sol)) :
        if (sol[i][0] - x)**2 + (sol[i][1] - y)**2 > distance :
             distance = (sol[i][0] - x)**2 + (sol[i][1] - y)**2
        if sol == [] :
            distance = 200000
    return distance

In [223]:
def list_3_ellipses(b, rssi):
    # on entre en entree un vecteur 
    # on fait une liste contenant les 3 ellipse
    liste_ellipses = []
    for (i, j) in itertools.combinations(range(0,3), 2):
        liste_ellipses += [ellipse_base(b[i], b[j], rssi[i], rssi[j])]
    return liste_ellipses

In [224]:
def sum_disdances(X, liste_ellipses):
    result = 0
    for i in range(0,3):
        result += distance(liste_ellipses[i], X)
    print(X, result)
    return result

In [225]:
def get_coordinates(b1, b2, b3, rssi1, rssi2, rssi3):
    b = [b1, b2, b3]
    rssi = [rssi1, rssi2, rssi3]
    liste_ellipses = list_3_ellipses(b, rssi)
    X = minimize(sum_disdances, [40, -100], method='Nelder-Mead', 
                 args = (liste_ellipses), 
                 options={'xtol': 1e-5}).x
    return X

In [226]:
%time
b1 = [39.797969, -105.073460]
b2 = [39.723151, -104.956216]
b3 = [39.732045, -104.973651]
rssi1 = -132
rssi2 = -120
rssi3 = -100
get_coordinates(b1, b2, b3, rssi1, rssi2, rssi3)
#distance(ellipse_base(b1, b2, rssi1,  rssi2), [39.732045, -104.973651])

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 4.77 µs
2*x_2 - 80.0
x_2**2 + 73.720270909091*x_2 + y_2**2 - 193.174363636364*y_2 + 1370.318625
2*x_2 - 80.0
x_2**2 + 60.6120931818182*x_2 + y_2**2 - 159.674792613636*y_2 + 473.107534125
2*x_2 - 80.0
x_2**2 + 66.131135*x_2 + y_2**2 - 174.781735*y_2 + 717.614961
[  40. -100.] 171984.969917969
2*x_2 - 84.0
x_2**2 + 73.720270909091*x_2 + y_2**2 - 193.174363636364*y_2 + 1370.318625
2*x_2 - 84.0
x_2**2 + 60.6120931818182*x_2 + y_2**2 - 159.674792613636*y_2 + 473.107534125
2*x_2 - 84.0
x_2**2 + 66.131135*x_2 + y_2**2 - 174.781735*y_2 + 717.614961
[  42. -100.] 167680.509121356
2*x_2 - 80.0
x_2**2 + 73.720270909091*x_2 + y_2**2 - 193.174363636364*y_2 + 1370.318625
2*x_2 - 80.0
x_2**2 + 60.6120931818182*x_2 + y_2**2 - 159.674792613636*y_2 + 473.107534125
2*x_2 - 80.0
x_2**2 + 66.131135*x_2 + y_2**2 - 174.781735*y_2 + 717.614961
[  40. -105.] 179232.385654037
2*x_2 - 84.0
x_2**2 + 73.720270909091*x_2 + y_2**2 - 193.174363636364*y_2 + 1370.

2*x_2 - 126.28125
x_2**2 + 66.131135*x_2 + y_2**2 - 174.781735*y_2 + 717.614961
[ 63.140625 -39.84375 ] 0
2*x_2 - 126.84375
x_2**2 + 73.720270909091*x_2 + y_2**2 - 193.174363636364*y_2 + 1370.318625
2*x_2 - 126.84375
x_2**2 + 60.6120931818182*x_2 + y_2**2 - 159.674792613636*y_2 + 473.107534125
2*x_2 - 126.84375
x_2**2 + 66.131135*x_2 + y_2**2 - 174.781735*y_2 + 717.614961
[ 63.421875 -39.53125 ] 0
2*x_2 - 125.1875
x_2**2 + 73.720270909091*x_2 + y_2**2 - 193.174363636364*y_2 + 1370.318625
2*x_2 - 125.1875
x_2**2 + 60.6120931818182*x_2 + y_2**2 - 159.674792613636*y_2 + 473.107534125
2*x_2 - 125.1875
x_2**2 + 66.131135*x_2 + y_2**2 - 174.781735*y_2 + 717.614961
[ 62.59375 -41.5625 ] 0
2*x_2 - 126.4296875
x_2**2 + 73.720270909091*x_2 + y_2**2 - 193.174363636364*y_2 + 1370.318625
2*x_2 - 126.4296875
x_2**2 + 60.6120931818182*x_2 + y_2**2 - 159.674792613636*y_2 + 473.107534125
2*x_2 - 126.4296875
x_2**2 + 66.131135*x_2 + y_2**2 - 174.781735*y_2 + 717.614961
[ 63.21484375 -40.0390625 ] 0
2*x_

[ 62.88031006 -41.2310791 ] 0
2*x_2 - 125.754150390625
x_2**2 + 73.720270909091*x_2 + y_2**2 - 193.174363636364*y_2 + 1370.318625
2*x_2 - 125.754150390625
x_2**2 + 60.6120931818182*x_2 + y_2**2 - 159.674792613636*y_2 + 473.107534125
2*x_2 - 125.754150390625
x_2**2 + 66.131135*x_2 + y_2**2 - 174.781735*y_2 + 717.614961
[ 62.8770752  -41.23901367] 0
2*x_2 - 125.758544921875
x_2**2 + 73.720270909091*x_2 + y_2**2 - 193.174363636364*y_2 + 1370.318625
2*x_2 - 125.758544921875
x_2**2 + 60.6120931818182*x_2 + y_2**2 - 159.674792613636*y_2 + 473.107534125
2*x_2 - 125.758544921875
x_2**2 + 66.131135*x_2 + y_2**2 - 174.781735*y_2 + 717.614961
[ 62.87927246 -41.23657227] 0
2*x_2 - 125.74560546875
x_2**2 + 73.720270909091*x_2 + y_2**2 - 193.174363636364*y_2 + 1370.318625
2*x_2 - 125.74560546875
x_2**2 + 60.6120931818182*x_2 + y_2**2 - 159.674792613636*y_2 + 473.107534125
2*x_2 - 125.74560546875
x_2**2 + 66.131135*x_2 + y_2**2 - 174.781735*y_2 + 717.614961
[ 62.87280273 -41.25244141] 0
2*x_2 - 125.7

[ 62.87506676 -41.24979019] 0
2*x_2 - 125.749931335449
x_2**2 + 73.720270909091*x_2 + y_2**2 - 193.174363636364*y_2 + 1370.318625
2*x_2 - 125.749931335449
x_2**2 + 60.6120931818182*x_2 + y_2**2 - 159.674792613636*y_2 + 473.107534125
2*x_2 - 125.749931335449
x_2**2 + 66.131135*x_2 + y_2**2 - 174.781735*y_2 + 717.614961
[ 62.87496567 -41.25003815] 0
2*x_2 - 125.750082969666
x_2**2 + 73.720270909091*x_2 + y_2**2 - 193.174363636364*y_2 + 1370.318625
2*x_2 - 125.750082969666
x_2**2 + 60.6120931818182*x_2 + y_2**2 - 159.674792613636*y_2 + 473.107534125
2*x_2 - 125.750082969666
x_2**2 + 66.131135*x_2 + y_2**2 - 174.781735*y_2 + 717.614961
[ 62.87504148 -41.24985218] 0
2*x_2 - 125.750032424927
x_2**2 + 73.720270909091*x_2 + y_2**2 - 193.174363636364*y_2 + 1370.318625
2*x_2 - 125.750032424927
x_2**2 + 60.6120931818182*x_2 + y_2**2 - 159.674792613636*y_2 + 473.107534125
2*x_2 - 125.750032424927
x_2**2 + 66.131135*x_2 + y_2**2 - 174.781735*y_2 + 717.614961
[ 62.87501621 -41.24991417] 0
2*x_2 - 12

array([ 62.875, -41.25 ])

In [140]:
lol[0][1]

37.8413517914956

In [209]:
x1 = Symbol('x_1')
y1 = Symbol('y_1')
x2 = Symbol('x_2')
y2 = Symbol('y_2')
sol = solve((x1 +3, x2 - x1 +2), x1, x2)

In [211]:
sol[x2]

-5

In [112]:
solve(diff((x2 - 2)**2 + (y2 - 3)**2, x2)

2*x_2 - 4

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

z = -np.linspace(9,15,100)
x = np.linspace(-26,26,1000)

x,z = np.meshgrid(x,z)

Z = -np.exp(-0.05*z) +4*(z+10)**2 
X = x**2


plt.contour(x,z,(X+Z),[0])
plt.xlim([-1.5,1.5])
plt.ylim([-11.5,-8.5])